In [6]:
import qlib
from qlib.constant import REG_CN
provider_uri = "../../data/qlib_data/cn_data_ak"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

[17128:MainThread](2022-09-01 21:59:46,966) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[17128:MainThread](2022-09-01 21:59:46,968) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[17128:MainThread](2022-09-01 21:59:46,968) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/PycharmProjects/QlibPlatform/data/qlib_data/cn_data_ak')}


In [7]:
# 导入QlibDataLoader
from qlib.data.dataset.loader import QlibDataLoader

In [10]:
# 加载原始特征，比如收盘价、最高价
qdl = QlibDataLoader(config=(['$close', '$high'],['close', 'high'])) 
qdl.load(instruments=['sh000300'], start_time='20190101', end_time='20191231') # 可以通过freq参数设置周期，默认freq='day'

,,close,high
datetime,instrument,,


In [12]:
from qlib.contrib.data.handler import Alpha158
from qlib.data.dataset import TSDatasetH
from qlib.contrib.model.pytorch_alstm_ts import ALSTM

# 配置数据
train_period = ("2015-01-01", "2019-12-31")
valid_period = ("2020-01-01", "2020-12-31")
test_period = ("2021-01-01", "2022-03-01")

dh = Alpha158(instruments='csi300', 
              start_time=train_period[0], 
              end_time=test_period[1],
              infer_processors={}
              
             )
ds = TSDatasetH(handler=dh,
                step_len=40, # 时间步数
                segments={"train": train_period, 
                          "valid": valid_period, 
                          "test": test_period})

[17128:MainThread](2022-09-01 22:01:16,085) INFO - qlib.timer - [log.py:117] - Time cost: 38.534s | Loading data Done
[17128:MainThread](2022-09-01 22:01:16,393) INFO - qlib.timer - [log.py:117] - Time cost: 0.248s | DropnaLabel Done
C:\Users\lyg\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
[17128:MainThread](2022-09-01 22:01:17,989) INFO - qlib.timer - [log.py:117] - Time cost: 1.596s | CSZScoreNorm Done
[17128:MainThread](2022-09-01 22:01:18,004) INFO - qlib.timer - [log.py:117] - Time cost: 1.917s | fit & process data Done
[17128:MainThread](2022-09-01 22:01:18,005) INFO - qlib.timer - [log.py:117] - Time cost: 40.454s | Init data Done


In [23]:
dh.fetch().isna().sum()

KMID       13620
KLEN       13620
KMID2      13620
KUP        13620
KUP2       13620
           ...  
VSUMD10    10221
VSUMD20     8297
VSUMD30     6915
VSUMD60     3977
LABEL0     14265
Length: 159, dtype: int64

In [27]:
dh.fetch().xs('2022-01-05')

,KMID,KLEN,KMID2,KUP,KUP2,KLOW,KLOW2,KSFT,KSFT2,OPEN0,...,VSUMN10,VSUMN20,VSUMN30,VSUMN60,VSUMD5,VSUMD10,VSUMD20,VSUMD30,VSUMD60,LABEL0
instrument,,,,,,,,,,,,,,,,,,,,,
SH600000,0.005514,0.009470,0.582277,0.003117,0.329116,0.000839,0.088607,0.003237,0.341768,0.994516,...,0.277560,0.453515,0.424346,0.477128,0.558550,0.444879,0.092969,0.151309,0.045744,0.011029
SH600009,0.006589,0.036519,0.180436,0.026913,0.736956,0.003017,0.082608,-0.017307,-0.473912,0.993454,...,0.367112,0.451288,0.450096,0.517636,0.456713,0.265777,0.097425,0.099808,-0.035271,0.018088
SH600010,-0.039315,0.052419,-0.750000,0.000000,0.000000,0.013105,0.250000,-0.026210,-0.500000,1.040923,...,0.472881,0.500820,0.593369,0.504476,0.382287,0.054238,-0.001641,-0.186738,-0.008952,0.023736
SH600011,-0.018120,0.040770,-0.444444,0.003775,0.092593,0.018875,0.462963,-0.003020,-0.074074,1.018454,...,0.499582,0.481340,0.447105,0.508786,0.316555,0.000837,0.037319,0.105789,-0.017573,-0.027354
SH600015,0.005711,0.008307,0.687504,0.002596,0.312496,0.000000,0.000000,0.003115,0.375009,0.994321,...,0.280007,0.439601,0.421741,0.470828,0.878217,0.439985,0.120798,0.156518,0.058343,0.006732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SZ300782,-0.025371,0.045559,-0.556890,0.015852,0.347951,0.004335,0.095159,-0.036888,-0.809682,1.026032,...,0.441461,0.522744,0.539570,0.526105,0.190049,0.117078,-0.045489,-0.079139,-0.052209,0.029285
SZ300866,-0.022162,0.041310,-0.536470,0.009428,0.228236,0.009720,0.235294,-0.021870,-0.529412,1.022664,...,0.563694,0.528693,0.552001,0.495146,-0.109966,-0.127388,-0.057385,-0.104001,0.009708,-0.026952
SZ300888,-0.025093,0.028578,-0.878048,0.000929,0.032521,0.002556,0.089431,-0.023467,-0.821138,1.025739,...,0.433643,0.464980,0.473871,0.503235,0.228781,0.132714,0.070041,0.052259,-0.006471,-0.022327
